In [ ]:
%pip install -r requirements.txt

In [1]:
# Tools create
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper



In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name 

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=5,doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]

In [7]:
# Custom tool creation
#from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaLLM , ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.agents import create_react_agent, AgentExecutor, AgentType, initialize_agent


In [6]:
llm = OllamaLLM(model="mistral:latest")
embeddings = OllamaEmbeddings(model="mistral:latest")

In [8]:
pdfdir = '/Users/giggso/Downloads/Books/PDF_Books/PDF_Test'

In [9]:
#loader=WebBaseLoader("https://docs.smith.langchain.com")

loader = PyPDFDirectoryLoader(pdfdir)
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200).split_documents(docs)
vectordb=Chroma.from_documents(documents=documents, embedding=embeddings,persist_directory="vecdb")

In [ ]:
vecdb = Chroma.from_documents(documents,embeddings)

In [11]:
retriever=vectordb.as_retriever()

In [13]:
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    return_source_documents=True
)

In [14]:
query = input("Enter your query: ")

In [15]:
result = qa_chain({"question": query})
print("Answer:", result["answer"])
print("Sources:", result["sources"])

/var/folders/q9/80fls_3j48l7f2r55v_g_8z40000gn/T/ipykernel_73145/1122491038.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query})


Answer:  AI governance refers to the systems and practices put in place to manage and control the use of artificial intelligence (AI) within an organization or society. This involves ensuring that AI systems are ethical, transparent, accountable, and secure. Key aspects of AI governance include:

1. Ensuring explainability of AI models: AI models should be designed to make their decision-making processes understandable by humans, allowing for oversight and auditability.
2. Establishing strong access controls and isolation around the training environment: This helps prevent abuse or interference in the development and deployment of AI models.
3. Maintaining accountability and auditability of model behavior: In complex black-box models, it may be difficult to determine how decisions are being made, but mechanisms need to be put in place for tracking model inputs, outputs, and decision logic.
4. Implementing bias monitoring and mitigation: AI systems can inherently perpetuate biases if th

In [16]:
from langchain.tools.retriever import create_retriever_tool

In [17]:

retriever_tool = create_retriever_tool(retriever,"docsearch","Search any info about the docs")
retriever_tool.name

'docsearch'

In [18]:
tools=[wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/giggso/Anaconda/anaconda3/envs/shared_proj/lib/python3.13/site-packages/wikipedia/__init__.py'>, top_k_results=5, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=5, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='docsearch', description='Search any info about the docs', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x119a151c0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x11d115550>, search_

In [19]:
# Create a more detailed prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI research assistant with expertise in analyzing and summarizing information.
    You have access to these tools: {tool_names}
    {tools}
    Approach:
    - Think like a human proficient in the tools
    - Use the tools to find the information needed
    - Summarize the information in your own words
    - Wikipedia for general knowledge
    - arXiv for scientific papers
    - Langsmith_Search for searching through specific documents
    
    Provide clear, concise answers and always cite your sources.
    When summarizing, keep it to 5 lines maximum."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


In [20]:
# Create the agent
agent = initialize_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION
)

/var/folders/q9/80fls_3j48l7f2r55v_g_8z40000gn/T/ipykernel_73145/3297062155.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [21]:
# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True  # To see the agent's thought process
)

In [22]:
# Usage
def get_response(question):
    response = agent_executor.invoke({
        "input": question,
        "chat_history": []
    })
    return response

In [25]:
# Modify the get_response function
def get_response(question):
    response = agent.invoke({
        "input": question,
        "chat_history": [],
    })
    return response["output"]

# Test the agent
try:
    question = input("Enter your question: ")
    result = get_response(question)
    print("\nResponse:", result)
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?list=search&srprop=&srlimit=5&limit=5&srsearch=List+of+deployment+models+for+large+language+models&format=json&action=query (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x11f7e9160>: Failed to resolve 'en.wikipedia.org' ([Errno 8] nodename nor servname provided, or not known)"))


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI research assistant. Use these tools:
    {tools}
    Available tool names: {tool_names}
    
    Give clear, concise answers and cite your sources."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)



In [ ]:
# Simple get_response function
def get_response(question):
    return agent_executor.invoke({
        "input": question,
        "chat_history": [HumanMessage(content=question)],
        "agent_scratchpad": [AIMessage(content="")]
    })

In [ ]:

# Test
question = input("Enter your question: ")
result = get_response(question)
print(result)


In [83]:

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant. Use these tools:
    {tools}
    
    Available tools: {tool_names}
    
    Follow this format:
    Thought: Consider what to do
    Action: Tool to use
    Action Input: Input for the tool
    Observation: Result
    ... (repeat if needed)
    Thought: Final conclusion
    Final Answer: Your response"""),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# Simple execution
def get_response(question):
    return agent_executor.invoke({"input": question})

# Test
question = "What is Langchain in 5 lines"
result = get_response(question)
print(result)




> Entering new AgentExecutor chain...


ValueError: variable agent_scratchpad should be a list of base messages, got  of type <class 'str'>